In [1]:
import numpy as np
import pandas as pd



In [48]:
class Simulation:
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_int_arr()   #time of next arrival
        self.t_arrival2=0                   #time of queue 2
        self.t_arrival3=0                   #time of queue 3
        self.t_departure1=float('inf')      #departure time from server 1
        self.t_departure2=float('inf')      #departure time from server 2
        self.t_departure3=float('inf')      #departure time from server 3
        self.dep_sum1=0                     #Sum of service times by teller 1
        self.dep_sum2=0                     #Sum of service times by teller 2
        self.dep_sum3=0                     #Sum of service times by teller 3
        self.state_T1=0                     #current state of server1 (binary)
        self.state_T2=0                     #current state of server2 (binary)
        self.state_T3=0                     #current state of server3 (binary)
        self.total_wait_time1=0.0           #total wait time of server 1
        self.total_wait_time2=0.0           #total wait time of server 2
        self.total_wait_time3=0.0           #total wait time of server 3
        self.num_in_q=0                     #current number in queue
        self.num_in_q2=0                    #current number in queue 2
        self.num_in_q3=0                    #current number in queue 3
        self.number_in_queue=0              #customers who had to wait in line(counter)
        self.num_of_departures1=0           #number of customers served by teller 1  
        self.num_of_departures2=0           #number of customers served by teller 2 
        self.num_of_departures3=0           #number of customers served by teller 3 
        self.lost_customers=0               #customers who left without service
        
    def time_adv(self):                                                       
        t_next_event=min(self.t_arrival,self.t_departure1) 
        t_next_event2=min(self.t_departure1,self.t_departure2)
        t_next_event3=min(self.t_departure2,self.t_departure3)
        self.total_wait_time1 += (self.num_in_q*(t_next_event-self.clock))
        self.total_wait_time2 += (self.num_in_q2*(t_next_event2-self.clock))
        self.total_wait_time3 += (self.num_in_q3*(t_next_event3-self.clock))
        self.clock=t_next_event
                
        if self.t_arrival<self.t_departure1:
            self.arrival()
        else:
            self.teller1()
            self.teller2()
            self.teller3()
        
    def arrival(self):              
        self.num_arrivals += 1
        self.number_in_queue += 1

        if self.num_in_q == 0:                                 #schedule next departure or arrival depending on state of servers
            if self.state_T1==1:
                self.num_in_q+=1
                self.t_arrival=self.clock+self.gen_int_arr()
                
                
            else:
                self.state_T1=1
                self.dep1= self.gen_service_time_teller1()
                self.dep_sum1 += self.dep1
                self.t_departure1=self.clock + self.dep1
                self.t_arrival=self.clock+self.gen_int_arr()

        else:       #if queue length is less than 4 generate next arrival and make customer join queue
            self.num_in_q+=1                           
            self.t_arrival=self.clock + self.gen_int_arr()
    
    def teller1(self):                #departure from server 1
        self.num_of_departures1 += 1
        if self.num_in_q>0:
            self.dep1= self.gen_service_time_teller1()
            self.dep_sum1 += self.dep1
            self.t_departure1=self.clock + self.dep1
            self.num_in_q-=1
            if self.num_in_q2 == 0:                                 #schedule next departure or arrival depending on state of servers
                if self.state_T2==1:
                    self.num_in_q2+=1
                    self.t_arrival2=self.t_departure1
                    


                else:
                    self.state_T2=1
                    self.t_arrival2=self.t_departure1
                    self.dep2= self.gen_service_time_teller2()
                    self.dep_sum2 += self.dep2
                    self.t_departure2=self.clock + self.dep2
                    
                    

            else:       #if queue length is less than 4 generate next arrival and make customer join queue
                self.num_in_q2+=1                           
                self.t_arrival2=self.t_departure1
                
        else:
            self.t_departure1=float('inf') 
            self.state_T1=0  
            if self.num_in_q2 == 0:                                 #schedule next departure or arrival depending on state of servers
                if self.state_T2==1:
                    self.num_in_q2+=1
                    self.t_arrival2=self.t_departure1
                    


                else:
                    self.state_T2=1
                    self.dep2= self.gen_service_time_teller2()
                    self.dep_sum2 += self.dep2
                    self.t_departure2=self.clock + self.dep2
                    self.t_arrival2=self.t_departure1
                    

            else:       #if queue length is less than 4 generate next arrival and make customer join queue
                self.num_in_q2+=1                           
                self.t_arrival2=self.t_departure1
                
    
    def teller2(self):                #departure from server 2
        self.num_of_departures2 += 1
        if self.num_in_q2>0:
            self.dep2= self.gen_service_time_teller2()
            self.dep_sum2 += self.dep2
            self.t_departure2=self.clock + self.dep2
            self.num_in_q2-=1
            if self.num_in_q3 == 0:                                 #schedule next departure or arrival depending on state of servers
                if self.state_T3==1:
                    self.num_in_q3+=1
                    self.t_arrival3=self.t_departure2
                    


                else:
                    self.state_T3=1
                    self.dep3= self.gen_service_time_teller3()
                    self.dep_sum3 += self.dep3
                    self.t_departure3=self.clock + self.dep3
                    self.t_arrival3=self.t_departure2
                    

            else:       #if queue length is less than 4 generate next arrival and make customer join queue
                self.num_in_q3+=1                           
                self.t_arrival3=self.t_departure2
                
        else:
            self.t_departure2=float('inf')
            self.state_T2=0
            if self.num_in_q3 == 0:                                 #schedule next departure or arrival depending on state of servers
                if self.state_T3==1:
                    self.num_in_q3+=1
                    self.t_arrival3=self.t_departure2
                    

                else:
                    self.state_T3=1
                    self.dep3= self.gen_service_time_teller3()
                    self.dep_sum3 += self.dep3
                    self.t_departure3=self.clock + self.dep3
                    self.t_arrival3=self.t_departure2
                    

            else:       #if queue length is less than 4 generate next arrival and make customer join queue
                self.num_in_q3+=1                           
                self.t_arrival3=self.t_departure2
                
    
    def teller3(self):                #departure from server 3
        if self.clock >self.dep_sum3:
            self.num_of_departures3 += 1
            if self.num_in_q3>0:
                self.dep3= self.gen_service_time_teller3()
                self.dep_sum3 += self.dep3
                self.t_departure3=self.clock + self.dep3
                self.num_in_q3-=1


            else:
                self.t_departure3=float('inf')
                self.state_T3=0

    
    def gen_int_arr(self):                                             #function to generate arrival times using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 10)
    
    def gen_service_time_teller1(self):                                #function to generate service time for teller 1 using inverse trnasform
        return (np.random.triangular(5, 8, 11))
    
    def gen_service_time_teller2(self):                                #function to generate service time for teller 2 using inverse trnasform
        return (np.random.uniform(2,6))
    
    def gen_service_time_teller3(self):                                #function to generate service time for teller 3 using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 15)





In [49]:
s = Simulation()
df = pd.DataFrame(columns=['Average interarrival time','Average service time teller1','Average service time teller 2','Average service time teller 3','Utilization teller 1','Utilization teller 2','Utilization teller 3','People who had to wait in line 1','People who had to wait in line 2', 'People who had to wait in line 3','Total average wait time for queue 1','Total average wait time for queue 2','Total average wait time for queue 3','a', 'b','c'])


for i in range(200):
    np.random.seed(i)
    s.__init__()
    while s.clock <= 10*48:
        s.time_adv() 
    a=pd.Series([s.clock/s.num_arrivals,s.dep_sum1/s.num_of_departures1,s.dep_sum2/s.num_of_departures2,s.dep_sum3/s.num_of_departures3, s.dep_sum1/s.clock,s.dep_sum2/s.clock,s.dep_sum3/s.clock,s.num_in_q,s.num_in_q2,s.num_in_q3,s.total_wait_time1/s.num_arrivals, s.total_wait_time2/s.num_arrivals,s.total_wait_time3/s.num_arrivals,s.num_of_departures1, s.num_of_departures2, s.num_of_departures3],index=df.columns)
    df=df.append(a,ignore_index=True) 

df

/Users/kenneth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars
/Users/kenneth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


,Average interarrival time,Average service time teller1,Average service time teller 2,Average service time teller 3,Utilization teller 1,Utilization teller 2,Utilization teller 3,People who had to wait in line 1,People who had to wait in line 2,People who had to wait in line 3,Total average wait time for queue 1,Total average wait time for queue 2,Total average wait time for queue 3,a,b,c
0,8.619021,8.143140,4.017504,18.509857,0.927916,0.457797,0.920382,0.0,0.0,30.0,13.440404,NaN,NaN,55.0,55.0,24.0
1,8.141271,8.142522,4.052976,15.904502,0.966250,0.480955,0.927116,1.0,0.0,28.0,24.659094,NaN,NaN,57.0,57.0,28.0
2,7.312414,8.090952,4.005880,15.940526,0.989115,0.489717,0.957846,6.0,0.0,29.0,30.214451,NaN,NaN,59.0,59.0,29.0
3,9.254270,8.005652,4.166991,12.229694,0.865077,0.450278,0.864070,0.0,0.0,17.0,11.891442,NaN,NaN,52.0,52.0,34.0
4,13.081608,8.034172,3.997191,13.887328,0.614158,0.305558,0.860750,0.0,0.0,6.0,5.322404,NaN,NaN,37.0,37.0,30.0
5,10.701074,7.911306,4.046822,23.245053,0.739300,0.378170,0.917158,0.0,0.0,25.0,8.049625,NaN,NaN,45.0,45.0,19.0
6,11.134226,8.048103,3.941231,15.580824,0.706398,0.345930,0.890504,0.0,0.0,14.0,6.219630,NaN,NaN,43.0,43.0,28.0
7,8.326881,7.970636,3.930773,16.147650,0.957217,0.472058,0.869305,0.0,0.0,31.0,23.171000,NaN,NaN,58.0,58.0,26.0
8,10.509488,8.339658,3.899585,10.989373,0.707282,0.330722,0.909271,4.0,0.0,0.0,8.432250,NaN,NaN,41.0,41.0,40.0
9,11.028991,8.001028,4.067026,13.595252,0.725454,0.368758,0.896497,0.0,0.0,11.0,7.482588,NaN,NaN,44.0,44.0,32.0


In [39]:
np.average(df['Utilization teller 1'])

0.7836115857239911

In [40]:
np.average(df['Utilization teller 2'])

0.38442293322201765

In [41]:
np.average(df['Utilization teller 3'])

0.989766165916582

In [42]:
np.average(df['People who had to wait in line 1'])

1.58

In [43]:
np.average(df['People who had to wait in line 3'])

14.615